In [4]:
import praw

import csv
from datetime import datetime
import pandas as pd
import datetime as dt
import pickle
import numpy as np

import re
import string

import nltk
from nltk import word_tokenize, pos_tag


In [ ]:
def setup(id, secret, password, user_agent, username):
    """Sets up the praw api """
    reddit = praw.Reddit(client_id = id,
                     client_secret = secret,
                     password = password,
                     user_agent = user_agent,
                     username = username)
    return reddit

In [7]:
def get_posts(subreddit, count):
    """Scrapes Reddit submissions
    type subreddit: string, name of the subreddit
    type count : int, maximum number of posts to crawl
    """
    filename = 'ControversialPostsFuturology.csv'
   
    writer = csv.writer(open(filename, 'wt', encoding = 'utf-8'))
    #the following attributes were identified and crawled for the each post object
    writer.writerow(['idstr', 'created_datetime', 'flair_text', 'flair_css_class',
                     'author', 'title', 'selftext', 
                     'distinguished', 'textlen']) 
    item_count = 0
    comment_count = 0
    for submission in reddit.subreddit(subreddit).controversial(limit=None): 
        try:
            item_count += 1
            idstr = submission.id
            created = submission.created
            #convert the utc field into readable datetime format for Dynamic Topic Modelling
            created_datetime = datetime.fromtimestamp(created).strftime('%Y' + '-' + '%m' + '-' + '%d')
            flair_text = submission.link_flair_text
            flair_css_class = submission.link_flair_css_class
            author = submission.author
            title = submission.title
            selftext = submission.selftext
            distinguished = submission.distinguished
            textlen = len(submission.selftext)
            writer.writerow((idstr, created_datetime, flair_text, flair_css_class,
                             author, title, selftext,
                             distinguished, textlen))
            if item_count == count:
                break
        except:
            print("Error in fetching the posts")

In [8]:
get_posts('Futurology', 1000)

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

DATA CLEANING

In [1]:
def pos_filter(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives'''
    is_noun_or_adj = lambda pos: pos[:2] in ['NN', 'JJ', 'CD', 'NNP', 'RB', 'VB']
    tokenized = word_tokenize(text)
    all_nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_or_adj(pos)] 
    return ' '.join(all_nouns_adj)

In [2]:
def clean_text(text):
    my_punctuation = string.punctuation
    my_punctuation += '’' #frequent occurence in data
    my_stopwords = nltk.corpus.stopwords.words('english')
    
    text_list = []
    
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('…', '', text)
    text = re.sub('['+my_punctuation +']+', ' ', text)

    for word in text.split(' '):
        if word in my_stopwords or len(word) == 1:
            word = ""
            continue
        text_list.append(word)
    text = ' '.join(text_list)
    return text

In [5]:
filename = 'ControversialPostsFuturology.csv'
#Replace empty selftext field with title field
with open(filename, 'r') as f:
    df = pd.read_csv(filename)
    df['selftext'] = np.where(df['selftext'].isna(), df['title'], df['selftext'])
df.to_csv('ControversialPostsFuturology.csv')

In [6]:
#Apply data cleaning functions to the attribute of interest - note that pos_tagging has to be done before cleaning because the former takes the letter case and punctuation marks into account for tagging
df['selftext'] = df['selftext'].apply(pos_filter)
df['selftext'] = df['selftext'].apply(clean_text)

In [7]:
#Save the clean dataframe
df.to_csv('CleanedControversialPostsFuturology.csv')